In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
pd.set_option('display.max_columns', None)
#显示所有行
pd.set_option('display.max_rows', None)

In [ ]:
train = pd.read_csv('/kaggle/input/riiid-test-answer-prediction/train.csv',
                   usecols=[ 2, 3, 4, 7],
                   dtype={
                          'user_id': 'int32',
                          'content_id': 'int16',
                          'content_type_id': 'int8',                          
                          'answered_correctly':'int8',
                          }
                   )

In [ ]:
lectures = pd.read_csv('../input/get-train-file/lectures.csv')
# lectures['content_type_id'] = 1
# lectures = lectures.rename(columns={'lecture_id':'content_id','tag':'lecture_tag','part':'lecture_part'})
# lectures['type_of'] = lectures['type_of'].replace('solving question', 'solving_question')

# lecture_tag_count =  lectures['lecture_tag'].value_counts().to_dict()
# lectures['lecture_tag'] = lectures['lecture_tag'].map(lecture_tag_count)

# # lectures['lecture_tag'] = lectures['lecture_tag'].apply(lambda x : 6 if ((x==7)|(x==5)) else x)

# lectures = pd.get_dummies(lectures, columns=['lecture_part','lecture_tag','type_of'])

In [ ]:
lectures_column = lectures.columns.to_list()
lectures_column.remove('content_id')
lectures_column.remove('content_type_id')
#获取lecture列名

In [ ]:
user_list = list(train['user_id'].unique())#获取所有用户名

In [ ]:
len(list(train['user_id'].unique()))

In [ ]:
user_info = pd.DataFrame(columns = ['user_id', 'lecture_part_1', 'lecture_part_2', 'lecture_part_3',
       'lecture_part_4', 'lecture_part_5', 'lecture_part_6', 'lecture_part_7',
       'lecture_tag_1', 'lecture_tag_2', 'lecture_tag_3', 'lecture_tag_4','lecture_tag_5',
       'lecture_tag_6','lecture_tag_7', 'type_of_concept', 'type_of_intention',
       'type_of_solving_question', 'type_of_starter', 'answered_cumsum',
       'answered_count'])


i=0
while True:
    user = user_list[i:i+50000]
    if len(user)==0:
        break
    temp = train[train['user_id'].isin(user)]    
    temp = pd.merge(temp,lectures, on = ['content_id','content_type_id'], how='left')
    temp[lectures_column] = temp[lectures_column].fillna(0)
    temp_group = temp.groupby(['user_id'])[lectures_column].agg('sum').reset_index()
    
    temp = temp[temp['content_type_id'] == 0].reset_index(drop=True)
    temp_group_count = temp[['user_id','answered_correctly']].groupby(['user_id']).agg(['sum','count']).reset_index(drop=True)
    temp_group_count.columns=['answered_cumsum','answered_count']

    temp_group[['answered_cumsum','answered_count']] = temp_group_count
    
    user_info = pd.concat([user_info,temp_group],axis = 0).reset_index(drop = True)
    i=i+50000
    print(i)

In [ ]:
user_info_columns = list(user_info.columns)

In [ ]:
user_info_columns.remove('user_id')

In [ ]:
for i in user_info_columns:
    user_info[i] = user_info[i].astype('int64')

In [ ]:
user_info['user_correctly_rate'] = user_info['answered_cumsum']/user_info['answered_count']
user_info['user_correctly_rate'] = user_info['user_correctly_rate'].mask((user_info['answered_count'] < 5), .65)


In [ ]:
user_info.to_csv('user_info.csv',index = False)

In [ ]:
user_info.head(100)